In [ ]:
print("Playground notebook")

In [ ]:
%notebook inline

In [ ]:
from pymatgen.electronic_structure.boltztrap2 import *
from monty.serialization import loadfn
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.electronic_structure.plotter import DosPlotter

In [ ]:
vrun = Vasprun('vasprun.xml',parse_projected_eigen=True)
data = VasprunBSLoader(vrun)

In [ ]:
# set curvature=False to speed up in case you do not need effective mass or hall coeficients
bztInterp = BztInterpolator(data,lpfac=10,energy_range=10,curvature=True, save_bztInterp=True,fname='bztInterp.json.gz')
# bztInterp = BztInterpolator(data,load_bztInterp=True,fname='bztInterp.json.gz')

# Compute and plot the band structure on standard symmetry path

In [ ]:
sbs = bztInterp.get_band_structure()
list(sbs.bands.values())[0].shape

In [ ]:
BSPlotter(sbs).show()

# On a specific kpoint path

In [ ]:
kpaths = [["Γ", "Y", "C0"], ["Σ0", "Γ", "Z", "A0"], ["E0", "T", "Y"], ["Γ", "S", "R", "Z", "T"], ["Γ", "Y'", "C0'"], ["Σ0'", "Γ", "Z'", "A0'"], ["E0'", "T'", "Y'"], ["Γ", "S'", "R'", "Z'", "T'"]]

kp_lbl = {"A0":np.array([0.5000000000, 0.5000000000, 0.5000000000]),
          "A0'":np.array([-0.5000000000, -0.5000000000, -0.5000000000]),
          "C0":np.array([-0.5000000000, 0.5000000000, 0.0000000000]),
          "C0'":np.array([0.5000000000, -0.5000000000, -0.0000000000]),
          "E0":np.array([-0.5000000000, 0.5000000000, 0.5000000000]),
          "E0'":np.array([0.5000000000, -0.5000000000, -0.5000000000]),
          "Γ":np.array([0.0000000000, 0.0000000000, 0.0000000000]),
          "R":np.array([0.0000000000, 0.5000000000, 0.5000000000]),
          "R'":np.array([-0.0000000000, -0.5000000000, -0.5000000000]),
          "S":np.array([0.0000000000, 0.5000000000, 0.0000000000]),
          "S'":np.array([-0.0000000000, -0.5000000000, -0.0000000000]),
          "Σ0":np.array([0.5000000000, 0.5000000000, 0.0000000000]),
          "Σ0'":np.array([-0.5000000000, -0.5000000000, -0.0000000000]),
          "T":np.array([-0.5000000000, 0.5000000000, 0.5000000000]),
          "T'":np.array([0.5000000000, -0.5000000000, -0.5000000000]),
          "Y":np.array([-0.5000000000, 0.5000000000, 0.0000000000]),
          "Y'":np.array([0.5000000000, -0.5000000000, -0.0000000000]),
          "Z":np.array([0.0000000000, 0.0000000000, 0.5000000000]),
          "Z'":np.array([-0.0000000000, -0.0000000000, -0.5000000000]),
          }
sbs = bztInterp.get_band_structure(kpaths,kp_lbl)
BSPlotter(sbs).show()

# Compute and plot Dos

In [ ]:
tot_dos = bztInterp.get_dos()
len(tot_dos.energies)

In [ ]:
# set progress=True to show a progress bar
tot_proj_dos = bztInterp.get_dos(partial_dos=True,progress=True)

In [ ]:
len(tot_proj_dos.get_spd_dos().values())

In [ ]:
pltdos = DosPlotter(sigma=0.02)
pltdos.add_dos_dict(tot_proj_dos.get_element_dos())
pltdos.show()

# Transport Properties

In [ ]:
bztTransp = BztTransportProperties(bztInterp,temp_r = np.arange(300,1300,300), doping=10.**np.arange(16,23), save_bztTranspProps=True,fname='bztTranspProps.json.gz')
# bztTransp = BztTransportProperties(bztInterp,load_bztTranspProps=True,fname='bztTranspProps.json.gz')

In [ ]:
print('\t'.join(['Temp', '\mu', 'rows', 'columns tensor']))
for p in bztTransp.Conductivity_mu, bztTransp.Seebeck_mu, bztTransp.Kappa_mu, \
         bztTransp.Effective_mass_mu, bztTransp.Power_Factor_mu, bztTransp.Carrier_conc_mu:
    print('\t'.join([str(i) for i in p.shape]))

In [ ]:
bztTransp.compute_properties_doping(doping=10.**np.arange(16,23))

In [ ]:
print('\t'.join(['Temp', 'Doping', 'rows', 'columns tensor']))
for p in bztTransp.Conductivity_doping, bztTransp.Seebeck_doping, bztTransp.Kappa_doping, \
         bztTransp.Carriers_conc_doping,bztTransp.Effective_mass_doping, bztTransp.Power_Factor_doping:
    print('\t'.join([str(i) for i in p['n'].shape]))

# Plotter

In [ ]:
bztPlotter = BztPlotter(bztTransp,bztInterp)

In [ ]:
bztPlotter.plot_props('C','mu','temp')

In [ ]:
bztPlotter.plot_props('S','mu','temp',temps=[600,900,1200])

In [ ]:
bztPlotter.plot_props('S','doping','temp', temps=[600,900,1200], dop_type='p').show()

In [ ]:
bztPlotter.plot_props('S','temp','doping',doping=[1e16,1e20], dop_type='n').show()

In [ ]:
bztPlotter.plot_props('K','mu','temp',temps=[600,900,1200]).show()

In [ ]:
bztPlotter.plot_props('H','mu','temp',temps=[600,900,1200]).show()

In [ ]:
bztPlotter.plot_props('Po','doping','temp',temps=[600,900,1200],dop_type='p').show()

In [ ]:
bztPlotter.plot_props('E','doping','temp').show()

In [ ]:
bztPlotter.plot_props('Ca','mu','temp',temps=[600,900,1200]).show()

In [ ]:
bztPlotter.plot_bands().show()

In [ ]:
bztPlotter.plot_dos(T=200).show()